<a href="https://colab.research.google.com/github/sfwtopoulos/stocks/blob/master/Stocks_seq_1st_for_testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import pandas as pd
import math
import warnings
import numpy as np
import time
import sys

import pandas_datareader as pdr
#%tensorflow_version 2.x
# Restart runtime using 'Runtime' -> 'Restart runtime...'
%tensorflow_version 1.x
import tensorflow as tf
import keras


# from keras.layers import LSTM
# from keras.models import Sequential
# from keras.layers.wrappers import TimeDistributed
# from keras.layers.wrappers import TimeDistributed
# from keras.layers.core import Dense, Activation, Dropout
# from keras.preprocessing.sequence import pad_sequences

from tensorflow.python.keras.layers import Dense, Activation, Dropout, Input
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import LSTM
from tensorflow.python.keras.layers.wrappers import TimeDistributed
from tensorflow.python.keras.layers.wrappers import TimeDistributed
from tensorflow.python.keras.preprocessing.sequence import pad_sequences


from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import seaborn as sns
import matplotlib.pyplot as plt

import os 

warnings.filterwarnings('ignore',category=FutureWarning)
warnings.filterwarnings('ignore', category=DeprecationWarning)

TensorFlow is already loaded. Please restart the runtime to change versions.


In [2]:
pip install talos

     |████████████████████████████████| 61kB 2.1MB/s 
     |████████████████████████████████| 109.2MB 49kB/s 
     |████████████████████████████████| 8.7MB 36.4MB/s 
     |████████████████████████████████| 378kB 45.7MB/s 
     |████████████████████████████████| 491kB 46.7MB/s 
     |████████████████████████████████| 3.2MB 40.4MB/s 
     |████████████████████████████████| 839kB 46.1MB/s 
  Created wheel for kerasplotlib: filename=kerasplotlib-0.1.6-cp36-none-any.whl size=3601 sha256=051abd1f874bfc33d28ccd1026b4f58c9ac1ebc3dcc6be383b83f4b7f7b91b68
  Stored in directory: /root/.cache/pip/wheels/9d/d3/8c/9503a22b0a38e8b21c70ad834e4606d209193443e5c709305d
  Created wheel for astetik: filename=astetik-1.9.9-cp36-none-any.whl size=56960 sha256=4fdcb0c28e37b879469977188699c5f53c51102db0d65cf21b4e8091fae891ef
  Stored in directory: /root/.cache/pip/wheels/ae/70/21/c475cd079ec401dd6e1b9b1d42b4c38554ce12679bfb214aad
  Created wheel for wrangle: filename=wrangle-0.6.7-cp36-none-any.whl size=49894 

In [0]:
!pip install talos -q

In [2]:
# from keras.optimizers import Adam, Nadam
# from keras.activations import softmax
# from keras.losses import categorical_crossentropy, logcosh
# from talos.utils import lr_normalizer
import talos
from tensorflow.python.keras.optimizers import Adam, Nadam
from tensorflow.python.keras.activations import softmax
from tensorflow.python.keras.losses import categorical_crossentropy, logcosh
from talos.utils import lr_normalizer

Using TensorFlow backend.


In [5]:
print(keras.__version__)
print(tf.__version__)
#print(tensorflow.compat.v2.__version__)

2.3.0
1.14.0


In [0]:
#Data import from git repo
url = 'https://raw.githubusercontent.com/sfwtopoulos/stocks/master/stocks_dataset/combined.csv'
#df1 = pd.read_csv(url, error_bad_lines=False)
dfstocks = pd.read_csv(url, sep=',')

In [7]:
#fix column names
dfstocks=dfstocks.rename({' AAPL':'Company', ' Close/Last':'Close', ' Volume':'Volume', ' Open':'Open', ' High':'High', ' Low':'Low'}, axis=1);
for col in dfstocks.columns: 
    print(col) 

Date
Close
Volume
Open
High
Low
Company


In [0]:
#drop $sign from values
dfstocks.Close=dfstocks['Close'].astype(str)
dfstocks.Close=dfstocks.Close.apply(lambda x: x.replace('$',''))
dfstocks.Open=dfstocks['Open'].astype(str)
dfstocks.Open=dfstocks.Open.apply(lambda x: x.replace('$',''))
dfstocks.High=dfstocks['High'].astype(str)
dfstocks.High=dfstocks.High.apply(lambda x: x.replace('$',''))
dfstocks.Low=dfstocks['Low'].astype(str)
dfstocks.Low=dfstocks.Low.apply(lambda x: x.replace('$',''))

In [9]:
#drop rows containing nan or header from the csv files
dfstocks=dfstocks[~dfstocks.Low.str.contains("nan")]
dfstocks=dfstocks[~dfstocks.Low.str.contains("Low")]
dfstocks=dfstocks[~dfstocks.Low.str.contains("N/A")]
dfstocks=dfstocks[~dfstocks.Volume.str.contains("N/A")]
dfstocks.reset_index(drop=True, inplace=True)
dfstocks.Date.count()

229269

In [10]:
#sort dataframe based on date and Company Name
dfstocks = dfstocks.sort_values(['Date', 'Company'])
dfstocks.head(100)

,Date,Close,Volume,Open,High,Low,Company
1761,01/02/2013,78.4328,139906732,79.1171,79.2857,77.3757,AAPL
4248,01/02/2013,35.12,13767660,34.92,35.4,34.1,ABBV
6025,01/02/2013,32.05,20266410,32.3,32.45,31.64,ABT
8512,01/02/2013,69.06,4039095,67.59,69.06,67.55,ACN
10999,01/02/2013,38.34,6483720,37.92,38.73,37.92,ADBE
...,...,...,...,...,...,...,...
10747,01/02/2014,59.29,2745895,59.06,59.53,58.94,ADBE
13234,01/02/2014,168.05,1268722,167.33,170.71,166.5937,AGN
15721,01/02/2014,50.71,9196092,50.81,51.3,50.47,AIG
18208,01/02/2014,53.55,1909106,54.09,54.45,53.5,ALL


In [11]:
#Converting Datatypes
dfstocks.Date=pd.to_datetime(dfstocks.Date)
dfstocks.Close=pd.to_numeric(dfstocks.Close)
dfstocks.Volume=pd.to_numeric(dfstocks.Volume)
dfstocks.Open=pd.to_numeric(dfstocks.Open)
dfstocks.High=pd.to_numeric(dfstocks.High)
dfstocks.Low=pd.to_numeric(dfstocks.Low)
dfstocks.dtypes

Date       datetime64[ns]
Close             float64
Volume              int64
Open              float64
High              float64
Low               float64
Company            object
dtype: object

In [12]:
#dfstocks[dfstocks['Company'].str.contains("AMZN")]
sub_split=dfstocks[dfstocks['Company'].str.contains("MDLZ")]
sub_split.isnull().values.any()
sub_split.isna().values.any()
sub_split = sub_split.sort_values(['Date'])
sub_split.reset_index(drop=True, inplace=True)
sub_split.shape[0]

2487

In [13]:
sub_split

,Date,Close,Volume,Open,High,Low,Company
0,2010-02-16,19.3633,54149150,19.1695,19.3633,18.9288,MDLZ
1,2010-02-17,19.1360,24833350,19.2898,19.3499,19.0157,MDLZ
2,2010-02-18,19.2564,20606450,19.1494,19.2564,19.0324,MDLZ
3,2010-02-19,19.3299,15497250,19.2229,19.4168,19.1494,MDLZ
4,2010-02-22,19.3499,27292690,19.3098,19.4435,19.1494,MDLZ
...,...,...,...,...,...,...,...
2482,2019-12-24,54.9900,1833763,54.9400,55.1800,54.7400,MDLZ
2483,2019-12-26,54.8500,3565549,54.9700,55.0741,54.7200,MDLZ
2484,2019-12-27,55.2800,4229449,54.9300,55.3300,54.8000,MDLZ
2485,2019-12-30,55.0000,3104187,54.9200,55.0700,54.5651,MDLZ


In [0]:
# prepare training and testing data sets for LSTM based sequence modeling
#def dimitris_function(time_series, scaling=True,split_ratio=0.9):
def dimitris_function(time_series,split_ratio=0.9):

    series1 = pd.Series(time_series[time_series.columns[0]])
    series2 = pd.Series(time_series[time_series.columns[1]])
    series3 = pd.Series(time_series[time_series.columns[2]])
    series4 = pd.Series(time_series[time_series.columns[3]])
    series5 = pd.Series(time_series[time_series.columns[4]])

    values1 = series1.values
    values1 = values1.reshape((len(values1), 1))
    values2 = series2.values
    values2 = values2.reshape((len(values2), 1))
    values3 = series3.values
    values3 = values3.reshape((len(values3), 1))
    values4 = series4.values
    values4 = values4.reshape((len(values4), 1))
    values5 = series5.values
    values5 = values5.reshape((len(values5), 1))

    #scaler2 --> Close Price Scaler
    scaler1 = MinMaxScaler(feature_range=(0, 1))
    time_series["scaled1"] = scaler1.fit_transform(values1)
    scaler2 = MinMaxScaler(feature_range=(0, 1))
    time_series["scaled2"]= scaler2.fit_transform(values2)
    scaler3 = MinMaxScaler(feature_range=(0, 1))
    time_series["scaled3"]= scaler3.fit_transform(values3)
    scaler4 = MinMaxScaler(feature_range=(0, 1))
    time_series["scaled4"]= scaler4.fit_transform(values4)
    scaler5 = MinMaxScaler(feature_range=(0, 1))
    time_series["scaled5"]= scaler5.fit_transform(values5)


    time_series = time_series[["scaled1","scaled2","scaled3","scaled4","scaled5"]].values

        
    split_ratio = int(len(time_series) * split_ratio)

    train = time_series[0:split_ratio]
    test = time_series[split_ratio:len(time_series)-1]
    
    # return train,test,scaler1,scaler2
    return train,test,scaler2


In [15]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
#def get_seq_model(hidden_units1=40, hidden_units2=65, hidden_units3=32, drop=0.2,input_shape=(1,1)):
def get_seq_model(x_train, y_train, x_val, y_val, params):  

    # create and fit the LSTM network
    model = Sequential()

    # samples*timesteps*features
    # https://keras.io/getting-started/sequential-model-guide/
    #https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0227222

    ######!!!!!#####!!!!!
    #(train_x.shape[1],5) where 5 is the number of features is equal to :
    #(train_x.shape[1],train_x.shape[2])
    model.add(LSTM(
                   input_shape=(x_train.shape[1],x_train.shape[2]),
                   units = params['hidden_1'],  
                   #activation='relu',recurrent_activation='sigmoid',
                   return_sequences=True
    ))
    model.add(Dropout(params['dropout']))
    model.add(LSTM(
                   units = params['hidden_2'],
                   #activation='relu',recurrent_activation='sigmoid',
                   return_sequences=True
    ))
    model.add(Dropout(params['dropout']))   
    model.add(LSTM(
                   units = params['hidden_3'], 
                   #activation='relu',recurrent_activation='sigmoid',
                   return_sequences=True
    ))
    model.add(Dropout(params['dropout']))

    # readout layer. TimeDistributedDense uses the same weights for all
    # time steps.
    # model.add(TimeDistributed(Dense(1))) #number of inputs in the 1st layer
   # model.add(TimeDistributed(Dense(5)))
    model.add(TimeDistributed(Dense(5)))
    start = time.time()
    
    model.compile(loss="mse", optimizer="rmsprop")
    timestr = time.strftime("%Y%m%d-%H%M%S")
    checkpoint_path = "/content/gdrive/My Drive/checkpoints1"  + "/cp-{epoch:04d}.ckpt"
    checkpoint_dir = os.path.dirname(checkpoint_path)
    cp_callback = tf.keras.callbacks.ModelCheckpoint(
        checkpoint_path, verbose=1, save_weights_only=True,#save_best_only='TRUE'
        # Save weights, every 5-epochs.
        period=15)
   # early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=40)
    
    out = model.fit(x_train, y_train,
              epochs=params['epochs'], batch_size=1,
              validation_data=[x_val, y_val],
              #epochs=200, batch_size=1,
              verbose=1,callbacks=[cp_callback])

    return out, model

In [0]:
#Parameters for the model
p = {'hidden_1':[40, 45],
     'hidden_2':[40, 45],
     'hidden_3':[40, 45],
     'dropout': (0.5, 0.6, 0.1),
     'epochs': [20, 30,100]
     }

In [0]:
# print(type(sub_split.Close))
sub_split_date=sub_split[['Date']]
sub_split=sub_split[['Volume', 'Close', 'Open', 'High', 'Low']]
#sub_split.columns[1]

In [19]:
# split train and test datasets
train,test,scaler_Close = dimitris_function(sub_split,split_ratio=0.7)

#train
print(len(train))
print(train.shape[0])
print(train.shape)
print(test.shape)
print(train)

1740
1740
(1740, 5)
(746, 5)
[[0.88815584 0.02021012 0.01335138 0.01636724 0.01466068]
 [0.39046289 0.01423411 0.01651313 0.01601441 0.01695744]
 [0.31870301 0.01739958 0.01282311 0.01355248 0.01739882]
 ...
 [0.07876951 0.68608191 0.69959394 0.69877535 0.69367717]
 [0.13961111 0.67241044 0.68067072 0.68508332 0.68019791]
 [0.10580105 0.69607261 0.66910654 0.69890701 0.68019791]]


In [20]:
#We use numpy to reshape our time series into 3D tensors.
train = np.reshape(train,(train.shape[0],1,5))
test = np.reshape(test,(test.shape[0],1,5))

train_x = train[:,:,:]
train_y = train[:,:,:]

test_x = test[:,:,:]
test_y = test[:,:,:]

print("Data Split Complete")

print("train_x shape={}".format(train_x.shape))
print("train_y shape={}".format(train_y.shape))
print("test_x shape={}".format(test_x.shape))
print("test_y shape={}".format(test_y.shape))

Data Split Complete
train_x shape=(1740, 1, 5)
train_y shape=(1740, 1, 5)
test_x shape=(746, 1, 5)
test_y shape=(746, 1, 5)


In [0]:
# x=np.concatenate((train_x, test_x), axis=1)
# y=np.concatenate((train_y, test_y), axis=1)
# print("x shape={}".format(x.shape))
# print("y shape={}".format(y.shape))
import talos


In [22]:
#####!!!!!#####!!!!!
#(train_x.shape[1],5) where 5 is the number of features is equal to :
train_x.shape[2]
 #,train_x.shape[2])
#input_shape=(train_x.shape[1],train_x.shape[2])
#input_shape.shape
#validation_data=[(test_x.shape[1],test_x.shape[2]), test_y]
#test=tuple(validation_data)

5

In [0]:
import sklearn

In [26]:
print('The scikit-learn version is {}.'.format(sklearn.__version__))
talos.Sca

The scikit-learn version is 0.22.2.post1.


In [30]:

scan_obj = talos.Scan(x=train_x,
                      y=train_y,
                      x_val=test_x,
                      #x_val=train_x,
                      y_val=test_y,
                     #y_val=train_y,

                      params=p,
                      val_split=0.1,
                      model=get_seq_model,
                      #shuffle=False,
                      experiment_name='gdrive/My Drive/test',
                      #fraction_limit=0.1
                      )


  0%|          | 0/24 [00:00<?, ?it/s]

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Train on 1740 samples, validate on 746 samples
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/20
1740/1740 [==============================] - 11s 6ms/sample - loss: 0.0204 - val_loss: 0.0066
Epoch 2/20
1740/1740 [==============================] - 9s 5ms/sample - loss: 0.0079 - val_loss: 0.0023
Epoch 3/20
1740/1740 [==============================] - 9s 5ms/sample - loss: 0.0058 - val_loss: 0.0072
Epoch 4/20
1740/1740 [==============================] - 9s 5ms/sample - loss: 0.0050 - val_loss: 0.0017
Epoch 5/20
1740/1740 [==============================] - 9s 5ms/sample - loss: 0.0049 - val_loss: 0.0029
Epoch 6/20
1740/1740 [==============================] - 9s 5ms/sample - loss: 0.0049 - val_loss: 0.0022
Epoch 7/20
1740/1740 [==============================] - 9s 5ms/sample - loss: 0.0048 - val_loss: 0.0037
Epoch 8/20
1

  4%|▍         | 1/24 [03:14<1:14:32, 194.48s/it]

Train on 1740 samples, validate on 746 samples
Epoch 1/20
1740/1740 [==============================] - 11s 6ms/sample - loss: 0.0195 - val_loss: 0.0014
Epoch 2/20
1740/1740 [==============================] - 9s 5ms/sample - loss: 0.0071 - val_loss: 0.0043
Epoch 3/20
1740/1740 [==============================] - 9s 5ms/sample - loss: 0.0056 - val_loss: 0.0025
Epoch 4/20
1740/1740 [==============================] - 9s 5ms/sample - loss: 0.0051 - val_loss: 0.0028
Epoch 5/20
1740/1740 [==============================] - 9s 5ms/sample - loss: 0.0047 - val_loss: 0.0014
Epoch 6/20
1740/1740 [==============================] - 9s 5ms/sample - loss: 0.0048 - val_loss: 0.0015
Epoch 7/20
1740/1740 [==============================] - 10s 5ms/sample - loss: 0.0048 - val_loss: 0.0018
Epoch 8/20
1740/1740 [==============================] - 9s 5ms/sample - loss: 0.0047 - val_loss: 0.0041
Epoch 9/20
1740/1740 [==============================] - 9s 5ms/sample - loss: 0.0046 - val_loss: 0.0055
Epoch 10/20
174

  8%|▊         | 2/24 [06:26<1:11:04, 193.86s/it]

...

 92%|█████████▏| 22/24 [2:39:12<28:43, 861.78s/it]

Train on 1740 samples, validate on 746 samples
Epoch 1/100
1740/1740 [==============================] - 11s 6ms/sample - loss: 0.0201 - val_loss: 0.0014
Epoch 2/100
1740/1740 [==============================] - 9s 5ms/sample - loss: 0.0073 - val_loss: 0.0013
Epoch 3/100
1740/1740 [==============================] - 9s 5ms/sample - loss: 0.0054 - val_loss: 0.0016
Epoch 4/100
1740/1740 [==============================] - 9s 5ms/sample - loss: 0.0049 - val_loss: 0.0055
Epoch 5/100
1740/1740 [==============================] - 9s 5ms/sample - loss: 0.0046 - val_loss: 0.0013
Epoch 6/100
1740/1740 [==============================] - 9s 5ms/sample - loss: 0.0048 - val_loss: 0.0031
Epoch 7/100
1740/1740 [==============================] - 9s 5ms/sample - loss: 0.0047 - val_loss: 0.0093
Epoch 8/100
1740/1740 [==============================] - 9s 5ms/sample - loss: 0.0045 - val_loss: 0.0074
Epoch 9/100
1740/1740 [==============================] - 9s 5ms/sample - loss: 0.0045 - val_loss: 0.0040
Epoch 1

 96%|█████████▌| 23/24 [2:55:07<14:49, 889.97s/it]

Train on 1740 samples, validate on 746 samples
Epoch 1/100
1740/1740 [==============================] - 11s 7ms/sample - loss: 0.0206 - val_loss: 0.0085
Epoch 2/100
1740/1740 [==============================] - 9s 5ms/sample - loss: 0.0072 - val_loss: 0.0087
Epoch 3/100
1740/1740 [==============================] - 9s 5ms/sample - loss: 0.0055 - val_loss: 0.0022
Epoch 4/100
1740/1740 [==============================] - 9s 5ms/sample - loss: 0.0047 - val_loss: 0.0022
Epoch 5/100
1740/1740 [==============================] - 9s 5ms/sample - loss: 0.0048 - val_loss: 0.0048
Epoch 6/100
1740/1740 [==============================] - 9s 5ms/sample - loss: 0.0050 - val_loss: 0.0065
Epoch 7/100
1740/1740 [==============================] - 9s 5ms/sample - loss: 0.0044 - val_loss: 0.0049
Epoch 8/100
1740/1740 [==============================] - 9s 5ms/sample - loss: 0.0045 - val_loss: 0.0028
Epoch 9/100
1740/1740 [==============================] - 10s 6ms/sample - loss: 0.0046 - val_loss: 0.0051
Epoch 

100%|██████████| 24/24 [3:10:38<00:00, 476.62s/it]


In [0]:
#from google.colab import drive
#drive.mount('/content/drive')

In [0]:
scan_obj.data.head()

In [0]:
train_y

In [0]:
# train the model
#https://medium.com/@rajath95/how-to-save-and-upload-deep-learning-machine-learning-models-in-google-colab-using-google-drive-eb2901937ae
# seq_lstm_model.fit(train_x, train_y,
#                 epochs=2, batch_size=1,
#                 #epochs=200, batch_size=1,
#                 verbose=1)
# print("Model Fit Complete")

In [0]:
# train fit performance
trainPredict = seq_lstm_model.predict(train_x)
trainScore = math.sqrt(mean_squared_error(train_y[0], trainPredict[0]))
print('Train Score: %.2f RMSE' % (trainScore))

In [0]:
print()

In [0]:
trainPredict

In [0]:
# Pad input sequence
testPredict = pad_sequences(test_x,maxlen=train_x.shape[1],padding='post',dtype='float64')

In [0]:
# forecast values
testPredict = seq_lstm_model.predict(testPredict)
testPredict

In [0]:
#Drop all other variables except Close Price Output
testPredict=np.delete(testPredict, [2,3,4], 2)
trainPredict=np.delete(trainPredict, [2,3,4], 2)
# testPredict=np.delete(testPredict, 3, 2)
# trainPredict=np.delete(trainPredict, 3, 2)
# testPredict=np.delete(testPredict, 4, 2)
# trainPredict=np.delete(trainPredict, 4, 2)
testPredict=np.delete(testPredict, 0, 2)
trainPredict=np.delete(trainPredict, 0, 2)

In [0]:
testPredict

In [0]:
trainPredict


In [0]:
type(testPredict)

In [0]:
# # inverse transformation of the Close Price
trainPredict = scaler_Close.inverse_transform(trainPredict.\
                                        reshape(-1,1))
testPredict = scaler_Close.inverse_transform(testPredict.\
                                       reshape(-1,1))

In [0]:
trainPredict

In [0]:
# plot the true and forecasted values
split_ratio = len(trainPredict)+1

plt.plot(sub_split_date.Date,
          sub_split.Close.values,c='black',
          alpha=0.3,label='True Data')

plt.plot(sub_split_date.Date[1:split_ratio],
          trainPredict,label='Training Fit',c='g')

plt.plot(sub_split_date.Date[split_ratio+1:],
          testPredict[:test_x.shape[1]],label='Testing Forecast')
plt.title('Forecast Plot')
plt.legend()
#plt.show()
plt.rcParams["figure.figsize"] = (20,10)
plt.show()